# Data

We obtained our datasets from the USDA Economic Research Service and from The Educational Opportunity Project at Stanford. These datasets will subsequently be referred to as the Food dataset and the Education dataset respectively. In this notebook, we prepare our analysis dataset by reading in the data and merging the two datasets.

In addition to these 2 datasets, we used a small dataset from the below github URL to convert state names to state abbreviations. We needed to perform this conversion to make merging on state and county possible.

In [ ]:
#@title [Hidden Code] State Abbreviation codes from https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive 
drive.mount('/content/gdrive')
education_df = pd.read_csv("gdrive/My Drive/731-Semester-Project/data/seda_county_pool_gcs_v30.csv")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Education Dataset

First, we import the Education dataset. The Educational Opportunity Project at Stanford has a variety of datasets to choose from. Because our Food dataset is county-level, we chose an Education dataset that also has county-level observations. We chose the GCS pooled version. The pooled version contains "pooled overall" observations which means they contain estimates that are averaged across grades, years, and subjects. Since we want to analyze county level educational performance we did not see a need to use data at a per subject per grade level of detail. GCS stands for grade cohort standardized estimates.

In [ ]:
education_df.head()

,countyid,countyname,fips,stateabb,subgroup,gap_est,tot_asmts,cellcount,mn_asmts,mn_avg_ol,mn_grd_ol,mn_coh_ol,mn_mth_ol,mn_avg_eb,mn_grd_eb,mn_coh_eb,mn_mth_eb,mn_avg_eb_se,mn_grd_eb_se,mn_coh_eb_se,mn_mth_eb_se,mn_avg_ol_se,mn_grd_ol_se,mn_coh_ol_se,mn_mth_ol_se
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091
1,1001,Autauga County,1,AL,asn,0,987,82,12.036585,6.959411,NaN,NaN,NaN,6.938545,NaN,NaN,NaN,0.132585,NaN,NaN,NaN,0.134268,NaN,NaN,NaN
2,1001,Autauga County,1,AL,blk,0,17213,96,179.302083,4.000924,0.990984,-0.047023,-0.353096,4.001221,0.986638,-0.045586,-0.333673,0.040307,0.028377,0.017289,0.078631,0.040381,0.029171,0.017608,0.080899
3,1001,Autauga County,1,AL,ecd,0,29288,84,348.666667,4.455159,1.035494,-0.001799,-0.349522,4.455382,1.029429,-0.000746,-0.322555,0.038907,0.028050,0.018347,0.074120,0.039059,0.028976,0.018917,0.076113
4,1001,Autauga County,1,AL,fem,0,31568,84,375.809524,5.819313,1.071826,-0.091138,-0.655916,5.814861,1.068969,-0.085387,-0.634774,0.039181,0.026747,0.016382,0.074575,0.039311,0.027666,0.016879,0.076284


From the Education dataset, we drop all of the observations that contain information split out by demographic subgroups. By selecting the all category, we retain only the estimates for all demographic groups combined. This enables our dataset to have a single observation per county which makes merging and analyzing the data more streamlined.

> Indented block



In [ ]:
# Only grab overall data per county
education_df = education_df[education_df.subgroup == "all"]
education_df.head()

,countyid,countyname,fips,stateabb,subgroup,gap_est,tot_asmts,cellcount,mn_asmts,mn_avg_ol,mn_grd_ol,mn_coh_ol,mn_mth_ol,mn_avg_eb,mn_grd_eb,mn_coh_eb,mn_mth_eb,mn_avg_eb_se,mn_grd_eb_se,mn_coh_eb_se,mn_mth_eb_se,mn_avg_ol_se,mn_grd_ol_se,mn_coh_ol_se,mn_mth_ol_se
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091
15,1003,Baldwin County,1,AL,all,0,213214,96,2220.979167,5.428993,0.832481,-0.051916,-0.611446,5.428188,0.837198,-0.048984,-0.593051,0.030970,0.022101,0.013229,0.060861,0.030998,0.022601,0.013487,0.061982
30,1005,Barbour County,1,AL,all,0,26521,96,276.260417,3.692509,0.956793,0.129474,-0.743482,3.695245,0.952161,0.123856,-0.717099,0.035750,0.025260,0.015315,0.069926,0.035792,0.026005,0.015718,0.071609
44,1007,Bibb County,1,AL,all,0,25678,96,267.479167,4.235791,0.996612,0.021188,-0.888118,4.237211,0.990590,0.020140,-0.846959,0.036446,0.025803,0.015509,0.071291,0.036493,0.026596,0.015929,0.073065
56,1009,Blount County,1,AL,all,0,71940,96,749.375000,4.689641,0.866955,0.091370,-1.011810,4.689768,0.867453,0.088930,-0.980552,0.032432,0.023091,0.013791,0.063648,0.032465,0.023661,0.014084,0.064929


## Food Dataset

Next, we import the Food dataset. This dataset contains food desert measurements at a census tract level of observation. 

In [ ]:
food_df = pd.read_excel("gdrive/My Drive/731-Semester-Project/data/Food_deserts_in_the_US.xlsx", sheet_name=2)
food_df.head()

,CensusTract,State,County,Urban,POP2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,LILATracts_halfAnd10,LILATracts_1And20,LILATracts_Vehicle,HUNVFlag,LowIncomeTracts,PovertyRate,MedianFamilyIncome,LA1and10,LAhalfand10,LA1and20,LATracts_half,LATracts1,LATracts10,LATracts20,LATractsVehicle_20,LAPOP1_10,LAPOP05_10,LAPOP1_20,LALOWI1_10,LALOWI05_10,LALOWI1_20,lapophalf,lapophalfshare,lalowihalf,lalowihalfshare,lakidshalf,lakidshalfshare,laseniorshalf,laseniorshalfshare,lawhitehalf,...,lasnap10,lasnap10share,lapop20,lapop20share,lalowi20,lalowi20share,lakids20,lakids20share,laseniors20,laseniors20share,lawhite20,lawhite20share,lablack20,lablack20share,laasian20,laasian20share,lanhopi20,lanhopi20share,laaian20,laaian20share,laomultir20,laomultir20share,lahisp20,lahisp20share,lahunv20,lahunv20share,lasnap20,lasnap20share,TractLOWI,TractKids,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,0,0,0,0,0,10.0,74750,1,1,1,1,1,0,0,0,1357.480940,1732.225468,1357.480940,322.084612,411.690754,322.084612,1732.225468,0.905976,411.690754,0.215319,466.426430,0.243947,198.828220,0.103990,1482.881417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,448,507,221,1622,217,14,0,14,45,44,26,112
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,0,0,0,0,0,18.2,51875,0,1,0,1,0,0,0,0,483.429683,1410.374828,483.429683,145.141418,475.306031,145.141418,1410.374828,0.649942,475.306031,0.219035,448.163512,0.206527,139.305390,0.064196,412.291349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763,606,214,888,1217,5,0,5,55,75,87,202
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,0,0,0,0,0,19.1,52905,1,1,1,1,1,0,0,0,1417.874893,2764.604126,1417.874893,696.636797,1350.176549,696.636797,2764.604126,0.819628,1350.176549,0.400290,744.891575,0.220839,346.203097,0.102640,2114.851375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1578,894,439,2576,647,17,5,11,117,87,108,120
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,0,0,0,0,0,3.3,68079,1,1,1,1,1,0,0,0,1363.466885,3651.061015,1363.466885,409.587836,1067.844499,409.587836,3651.061015,0.832435,1067.844499,0.243467,846.613783,0.193026,766.521491,0.174766,3395.061920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1241,1015,904,4086,193,18,4,11,74,85,19,82
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,0,0,0,1,0,8.5,77819,1,1,1,1,1,0,0,1,2643.095161,7778.396188,2643.095161,623.076555,1913.453614,623.076555,7778.396188,0.722496,1913.453614,0.177731,2309.475651,0.214516,839.885471,0.078013,6298.710335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2692,3162,1126,8666,1437,296,9,48,310,355,198,488


Since the Food dataset contains full state names and the Education dataset contains state abbreviations, we add a variable to the Food dataset that contains the state abbreviations. This will enable us to later merge the datasets by state. 

In [ ]:
food_df['stateabb'] = food_df.apply(lambda row: us_state_abbrev[row['State']], axis=1)

food_df['stateabb']

0        AL
1        AL
2        AL
3        AL
4        AL
         ..
72859    WY
72860    WY
72861    WY
72862    WY
72863    WY
Name: stateabb, Length: 72864, dtype: object

The county names in the Food dataset have the words "County" and "Parish" as part of the county name string. add a variable to the Food dataset that removes these words from the county name. This will enable us to later merge the datasets by county.

In [ ]:
def remove_county(row):
  return row['countyname'].replace(" County", "").replace(" Parish", "")

education_df['County'] = education_df.apply(remove_county, axis=1)
education_df['County']

0           Autauga
15          Baldwin
30          Barbour
44             Bibb
56           Blount
            ...    
39014    Sweetwater
39029         Teton
39041         Uinta
39055      Washakie
39065        Weston
Name: County, Length: 3134, dtype: object

## Merging

Before merging the datasets, we want to add identifiers to each variable name. We add the prefix 'Educ_' to all Education variable names and we add the prefix 'Food_' to all the Food variable names. This is done for all variables except for County and State because we will need to merge on these variable and in order to do so, they need to have the same name.

In [ ]:
education_df.columns = ["Educ_" + x for x in education_df.columns]

education_df['County'] = education_df['Educ_County']
education_df['stateabb'] = education_df['Educ_stateabb']

education_df.head()

,Educ_countyid,Educ_countyname,Educ_fips,Educ_stateabb,Educ_subgroup,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,Educ_mn_avg_eb,Educ_mn_grd_eb,Educ_mn_coh_eb,Educ_mn_mth_eb,Educ_mn_avg_eb_se,Educ_mn_grd_eb_se,Educ_mn_coh_eb_se,Educ_mn_mth_eb_se,Educ_mn_avg_ol_se,Educ_mn_grd_ol_se,Educ_mn_coh_ol_se,Educ_mn_mth_ol_se,Educ_County,County,stateabb
0,1001,Autauga County,1,AL,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL
15,1003,Baldwin County,1,AL,all,0,213214,96,2220.979167,5.428993,0.832481,-0.051916,-0.611446,5.428188,0.837198,-0.048984,-0.593051,0.030970,0.022101,0.013229,0.060861,0.030998,0.022601,0.013487,0.061982,Baldwin,Baldwin,AL
30,1005,Barbour County,1,AL,all,0,26521,96,276.260417,3.692509,0.956793,0.129474,-0.743482,3.695245,0.952161,0.123856,-0.717099,0.035750,0.025260,0.015315,0.069926,0.035792,0.026005,0.015718,0.071609,Barbour,Barbour,AL
44,1007,Bibb County,1,AL,all,0,25678,96,267.479167,4.235791,0.996612,0.021188,-0.888118,4.237211,0.990590,0.020140,-0.846959,0.036446,0.025803,0.015509,0.071291,0.036493,0.026596,0.015929,0.073065,Bibb,Bibb,AL
56,1009,Blount County,1,AL,all,0,71940,96,749.375000,4.689641,0.866955,0.091370,-1.011810,4.689768,0.867453,0.088930,-0.980552,0.032432,0.023091,0.013791,0.063648,0.032465,0.023661,0.014084,0.064929,Blount,Blount,AL


In [ ]:
food_df.columns = ["Food_" + x for x in food_df.columns]

food_df['County'] = food_df['Food_County']
food_df['stateabb'] = food_df['Food_stateabb']

food_df.head()

,Food_CensusTract,Food_State,Food_County,Food_Urban,Food_POP2010,Food_OHU2010,Food_GroupQuartersFlag,Food_NUMGQTRS,Food_PCTGQTRS,Food_LILATracts_1And10,Food_LILATracts_halfAnd10,Food_LILATracts_1And20,Food_LILATracts_Vehicle,Food_HUNVFlag,Food_LowIncomeTracts,Food_PovertyRate,Food_MedianFamilyIncome,Food_LA1and10,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,Food_LATracts1,Food_LATracts10,Food_LATracts20,Food_LATractsVehicle_20,Food_LAPOP1_10,Food_LAPOP05_10,Food_LAPOP1_20,Food_LALOWI1_10,Food_LALOWI05_10,Food_LALOWI1_20,Food_lapophalf,Food_lapophalfshare,Food_lalowihalf,Food_lalowihalfshare,Food_lakidshalf,Food_lakidshalfshare,Food_laseniorshalf,Food_laseniorshalfshare,Food_lawhitehalf,...,Food_lapop20share,Food_lalowi20,Food_lalowi20share,Food_lakids20,Food_lakids20share,Food_laseniors20,Food_laseniors20share,Food_lawhite20,Food_lawhite20share,Food_lablack20,Food_lablack20share,Food_laasian20,Food_laasian20share,Food_lanhopi20,Food_lanhopi20share,Food_laaian20,Food_laaian20share,Food_laomultir20,Food_laomultir20share,Food_lahisp20,Food_lahisp20share,Food_lahunv20,Food_lahunv20share,Food_lasnap20,Food_lasnap20share,Food_TractLOWI,Food_TractKids,Food_TractSeniors,Food_TractWhite,Food_TractBlack,Food_TractAsian,Food_TractNHOPI,Food_TractAIAN,Food_TractOMultir,Food_TractHispanic,Food_TractHUNV,Food_TractSNAP,Food_stateabb,County,stateabb
0,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,0,0,0,0,0,10.0,74750,1,1,1,1,1,0,0,0,1357.480940,1732.225468,1357.480940,322.084612,411.690754,322.084612,1732.225468,0.905976,411.690754,0.215319,466.426430,0.243947,198.828220,0.103990,1482.881417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,448,507,221,1622,217,14,0,14,45,44,26,112,AL,Autauga,AL
1,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,0,0,0,0,0,18.2,51875,0,1,0,1,0,0,0,0,483.429683,1410.374828,483.429683,145.141418,475.306031,145.141418,1410.374828,0.649942,475.306031,0.219035,448.163512,0.206527,139.305390,0.064196,412.291349,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763,606,214,888,1217,5,0,5,55,75,87,202,AL,Autauga,AL
2,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,0,0,0,0,0,19.1,52905,1,1,1,1,1,0,0,0,1417.874893,2764.604126,1417.874893,696.636797,1350.176549,696.636797,2764.604126,0.819628,1350.176549,0.400290,744.891575,0.220839,346.203097,0.102640,2114.851375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1578,894,439,2576,647,17,5,11,117,87,108,120,AL,Autauga,AL
3,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,0,0,0,0,0,3.3,68079,1,1,1,1,1,0,0,0,1363.466885,3651.061015,1363.466885,409.587836,1067.844499,409.587836,3651.061015,0.832435,1067.844499,0.243467,846.613783,0.193026,766.521491,0.174766,3395.061920,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1241,1015,904,4086,193,18,4,11,74,85,19,82,AL,Autauga,AL
4,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,0,0,0,1,0,8.5,77819,1,1,1,1,1,0,0,1,2643.095161,7778.396188,2643.095161,623.076555,1913.453614,623.076555,7778.396188,0.722496,1913.453614,0.177731,2309.475651,0.214516,839.885471,0.078013,6298.710335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2692,3162,1126,8666,1437,296,9,48,310,355,198,488,AL,Autauga,AL


We can now merge the data by County and State. Because the Food dataset has multiple observations per year, it can be seen in the table below that there is repeated education information per observation. We will handle this in a few steps.

In [ ]:
combined_df = pd.merge(education_df, food_df, on=["County", "stateabb"])

combined_df.head()

,Educ_countyid,Educ_countyname,Educ_fips,Educ_stateabb,Educ_subgroup,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,Educ_mn_avg_eb,Educ_mn_grd_eb,Educ_mn_coh_eb,Educ_mn_mth_eb,Educ_mn_avg_eb_se,Educ_mn_grd_eb_se,Educ_mn_coh_eb_se,Educ_mn_mth_eb_se,Educ_mn_avg_ol_se,Educ_mn_grd_ol_se,Educ_mn_coh_ol_se,Educ_mn_mth_ol_se,Educ_County,County,stateabb,Food_CensusTract,Food_State,Food_County,Food_Urban,Food_POP2010,Food_OHU2010,Food_GroupQuartersFlag,Food_NUMGQTRS,Food_PCTGQTRS,Food_LILATracts_1And10,Food_LILATracts_halfAnd10,Food_LILATracts_1And20,...,Food_lasnap10share,Food_lapop20,Food_lapop20share,Food_lalowi20,Food_lalowi20share,Food_lakids20,Food_lakids20share,Food_laseniors20,Food_laseniors20share,Food_lawhite20,Food_lawhite20share,Food_lablack20,Food_lablack20share,Food_laasian20,Food_laasian20share,Food_lanhopi20,Food_lanhopi20share,Food_laaian20,Food_laaian20share,Food_laomultir20,Food_laomultir20share,Food_lahisp20,Food_lahisp20share,Food_lahunv20,Food_lahunv20share,Food_lasnap20,Food_lasnap20share,Food_TractLOWI,Food_TractKids,Food_TractSeniors,Food_TractWhite,Food_TractBlack,Food_TractAsian,Food_TractNHOPI,Food_TractAIAN,Food_TractOMultir,Food_TractHispanic,Food_TractHUNV,Food_TractSNAP,Food_stateabb
0,1001,Autauga County,1,AL,all,0,72864,96,759.0,5.636815,1.048619,-0.09283,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL,1001020100,Alabama,Autauga,1,1912,693,0,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,448,507,221,1622,217,14,0,14,45,44,26,112,AL
1,1001,Autauga County,1,AL,all,0,72864,96,759.0,5.636815,1.048619,-0.09283,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL,1001020200,Alabama,Autauga,1,2170,743,0,181,0.083410,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,763,606,214,888,1217,5,0,5,55,75,87,202,AL
2,1001,Autauga County,1,AL,all,0,72864,96,759.0,5.636815,1.048619,-0.09283,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL,1001020300,Alabama,Autauga,1,3373,1256,0,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1578,894,439,2576,647,17,5,11,117,87,108,120,AL
3,1001,Autauga County,1,AL,all,0,72864,96,759.0,5.636815,1.048619,-0.09283,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL,1001020400,Alabama,Autauga,1,4386,1722,0,0,0.000000,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1241,1015,904,4086,193,18,4,11,74,85,19,82,AL
4,1001,Autauga County,1,AL,all,0,72864,96,759.0,5.636815,1.048619,-0.09283,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,Autauga,AL,1001020500,Alabama,Autauga,1,10766,4082,0,181,0.016812,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2692,3162,1126,8666,1437,296,9,48,310,355,198,488,AL


Since there is repeated county and state information in our newly merged dataset, we will now drop the excess columns.

In [ ]:
combined_df.drop(["Food_State", "Food_County", "Food_stateabb", "Educ_stateabb", "Educ_County", "Educ_countyname"], axis=1, inplace=True)
combined_df

,Educ_countyid,Educ_fips,Educ_subgroup,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,Educ_mn_avg_eb,Educ_mn_grd_eb,Educ_mn_coh_eb,Educ_mn_mth_eb,Educ_mn_avg_eb_se,Educ_mn_grd_eb_se,Educ_mn_coh_eb_se,Educ_mn_mth_eb_se,Educ_mn_avg_ol_se,Educ_mn_grd_ol_se,Educ_mn_coh_ol_se,Educ_mn_mth_ol_se,County,stateabb,Food_CensusTract,Food_Urban,Food_POP2010,Food_OHU2010,Food_GroupQuartersFlag,Food_NUMGQTRS,Food_PCTGQTRS,Food_LILATracts_1And10,Food_LILATracts_halfAnd10,Food_LILATracts_1And20,Food_LILATracts_Vehicle,Food_HUNVFlag,Food_LowIncomeTracts,Food_PovertyRate,Food_MedianFamilyIncome,...,Food_lasnap10,Food_lasnap10share,Food_lapop20,Food_lapop20share,Food_lalowi20,Food_lalowi20share,Food_lakids20,Food_lakids20share,Food_laseniors20,Food_laseniors20share,Food_lawhite20,Food_lawhite20share,Food_lablack20,Food_lablack20share,Food_laasian20,Food_laasian20share,Food_lanhopi20,Food_lanhopi20share,Food_laaian20,Food_laaian20share,Food_laomultir20,Food_laomultir20share,Food_lahisp20,Food_lahisp20share,Food_lahunv20,Food_lahunv20share,Food_lasnap20,Food_lasnap20share,Food_TractLOWI,Food_TractKids,Food_TractSeniors,Food_TractWhite,Food_TractBlack,Food_TractAsian,Food_TractNHOPI,Food_TractAIAN,Food_TractOMultir,Food_TractHispanic,Food_TractHUNV,Food_TractSNAP
0,1001,1,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,AL,1001020100,1,1912,693,0,0,0.000000,0,0,0,0,0,0,10.0,74750,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,448,507,221,1622,217,14,0,14,45,44,26,112
1,1001,1,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,AL,1001020200,1,2170,743,0,181,0.083410,0,0,0,0,0,0,18.2,51875,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,763,606,214,888,1217,5,0,5,55,75,87,202
2,1001,1,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,AL,1001020300,1,3373,1256,0,0,0.000000,0,0,0,0,0,0,19.1,52905,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1578,894,439,2576,647,17,5,11,117,87,108,120
3,1001,1,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,AL,1001020400,1,4386,1722,0,0,0.000000,0,0,0,0,0,0,3.3,68079,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1241,1015,904,4086,193,18,4,11,74,85,19,82
4,1001,1,all,0,72864,96,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,Autauga,AL,1001020500,1,10766,4082,0,181,0.016812,0,0,0,0,1,0,8.5,77819,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,2692,3162,1126,8666,1437,296,9,48,310,355,198,488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
combined_df.Educ_subgroup.unique()

array(['all'], dtype=object)

As discussed earlier, we want to combine observations so that we have a single observation per state-county pair. This will streamline analysis. To do so, we take the average of each attribute for each state-county pair. Recall we already have single observations per state-county pair in the Education dataset, so this will not affect that data. We have multiple observations per state-county pair in the Food dataset because the measurements are taken by census tract. Our reasoning behind taking the average across census tracts is: for a majority of the attributes taking the average makes sense. For the attributes whose values represent counts, the resulting attribute value will not represent the count of people in that county but rather an average of the census tracts in that county. For boolean type variables, the resulting average will represent a percentage (in decimal form) of how many tracts in that county had a value of 1 before combining.

Combining the data with the mean() function results in all string variables being dropped. The only important string variables we need are County and State. Therefore, we add them back in by using the information from an unmerged dataset and the county IDs.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

combined_df = combined_df.groupby("Educ_countyid").mean()

# Don't look at this please
combined_df['County'] = ""
combined_df['State'] = ""
for county_id in combined_df.index.values:
  combined_df['County'][county_id] = education_df.loc[education_df.Educ_countyid == county_id]['County'].to_list()[0]
  combined_df['State'][county_id] = education_df.loc[education_df.Educ_countyid == county_id]['stateabb'].to_list()[0]

warnings.filterwarnings("default")

Our resulting dataset can be seen below. We now have a single observation per state-county pair. There are 3057 observations in our dataset, meaning it covers 3,057 counties. In the US there are a total of 3,141 counties. Therefore our dataset contains information for 97% of the counties in the US.

In [ ]:
combined_df

,Educ_fips,Educ_gap_est,Educ_tot_asmts,Educ_cellcount,Educ_mn_asmts,Educ_mn_avg_ol,Educ_mn_grd_ol,Educ_mn_coh_ol,Educ_mn_mth_ol,Educ_mn_avg_eb,Educ_mn_grd_eb,Educ_mn_coh_eb,Educ_mn_mth_eb,Educ_mn_avg_eb_se,Educ_mn_grd_eb_se,Educ_mn_coh_eb_se,Educ_mn_mth_eb_se,Educ_mn_avg_ol_se,Educ_mn_grd_ol_se,Educ_mn_coh_ol_se,Educ_mn_mth_ol_se,Food_CensusTract,Food_Urban,Food_POP2010,Food_OHU2010,Food_GroupQuartersFlag,Food_NUMGQTRS,Food_PCTGQTRS,Food_LILATracts_1And10,Food_LILATracts_halfAnd10,Food_LILATracts_1And20,Food_LILATracts_Vehicle,Food_HUNVFlag,Food_LowIncomeTracts,Food_PovertyRate,Food_MedianFamilyIncome,Food_LA1and10,Food_LAhalfand10,Food_LA1and20,Food_LATracts_half,...,Food_lapop20,Food_lapop20share,Food_lalowi20,Food_lalowi20share,Food_lakids20,Food_lakids20share,Food_laseniors20,Food_laseniors20share,Food_lawhite20,Food_lawhite20share,Food_lablack20,Food_lablack20share,Food_laasian20,Food_laasian20share,Food_lanhopi20,Food_lanhopi20share,Food_laaian20,Food_laaian20share,Food_laomultir20,Food_laomultir20share,Food_lahisp20,Food_lahisp20share,Food_lahunv20,Food_lahunv20share,Food_lasnap20,Food_lasnap20share,Food_TractLOWI,Food_TractKids,Food_TractSeniors,Food_TractWhite,Food_TractBlack,Food_TractAsian,Food_TractNHOPI,Food_TractAIAN,Food_TractOMultir,Food_TractHispanic,Food_TractHUNV,Food_TractSNAP,County,State
Educ_countyid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,1.0,0.0,72864.0,96.0,759.000000,5.636815,1.048619,-0.092830,-0.435624,5.635463,1.044776,-0.089008,-0.410573,0.033545,0.023657,0.014153,0.065684,0.033581,0.024269,0.014469,0.067091,1.001021e+09,0.583333,4547.583333,1685.083333,0.0,37.916667,0.010711,0.166667,0.166667,0.083333,0.166667,0.333333,0.166667,13.858333,61082.916667,0.750000,0.833333,0.500000,0.583333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1557.666667,1217.750000,545.500000,3571.250000,803.583333,39.500000,2.666667,19.333333,111.250000,109.166667,90.083333,223.500000,Autauga,AL
1003,1.0,0.0,213214.0,96.0,2220.979167,5.428993,0.832481,-0.051916,-0.611446,5.428188,0.837198,-0.048984,-0.593051,0.030970,0.022101,0.013229,0.060861,0.030998,0.022601,0.013487,0.061982,1.003011e+09,0.451613,5879.516129,2360.645161,0.0,74.419355,0.013246,0.096774,0.096774,0.096774,0.064516,0.129032,0.225806,14.358065,60664.935484,0.483871,0.516129,0.419355,0.451613,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2041.193548,1351.548387,986.064516,5037.193548,551.774194,43.483871,2.870968,39.225806,204.967742,257.806452,72.322581,208.741935,Baldwin,AL
1005,1.0,0.0,26521.0,96.0,276.260417,3.692509,0.956793,0.129474,-0.743482,3.695245,0.952161,0.123856,-0.717099,0.035750,0.025260,0.015315,0.069926,0.035792,0.026005,0.015718,0.071609,1.005950e+09,0.222222,3050.777778,1091.111111,0.0,354.777778,0.088595,0.333333,0.333333,0.111111,0.222222,0.333333,0.777778,24.533333,43123.000000,0.555556,0.555556,0.222222,0.222222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1387.111111,668.333333,434.333333,1464.444444,1430.555556,11.888889,3.222222,12.666667,128.000000,154.111111,89.111111,233.000000,Barbour,AL
1007,1.0,0.0,25678.0,96.0,267.479167,4.235791,0.996612,0.021188,-0.888118,4.237211,0.990590,0.020140,-0.846959,0.036446,0.025803,0.015509,0.071291,0.036493,0.026596,0.015929,0.073065,1.007010e+09,0.000000,5728.750000,1988.250000,0.0,556.000000,0.065628,0.000000,0.000000,0.000000,0.250000,0.250000,1.000000,16.025000,43362.500000,0.000000

In [ ]:
combined_df.to_csv("gdrive/My Drive/731-Semester-Project/data/combined_data.csv")